In [1]:
# Import our custom module
from movielens_analysis import Movies, Ratings, Tags, Links, Tests

# Initialize the objects once
m = Movies('../datasets/movies.csv')
t = Tags('../datasets/tags.csv')
r = Ratings('../datasets/ratings.csv', m)
l = Links('../datasets/links.csv', '../datasets/movies.csv')

In [2]:
%timeit
# Running the internal tests
test_suite = Tests()
test_suite.test_links()
test_suite.test_movies()
test_suite.test_tags()
test_suite.test_ratings()
print("✅ All systems go! The engine is running perfectly.")

Testing Links...
Testing Movies...
Testing Tags...
Testing Ratings.Movies...
Testing Ratings.Users...
✅ All systems go! The engine is running perfectly.


# Links

## Deep Dive: Custom Data Extraction
### Question: Can we get specific details for a specific list of movies?
We use our `get_imdb` method to fetch the **Director**, **Budget**, and **Runtime** for "Toy Story" (ID 1) and "Jumanji" (ID 2).

In [3]:
%timeit
# Fetching specific fields for Movie IDs 1 (Toy Story) and 2 (Jumanji)
target_movies = ['1', '2']
target_fields = ['Director', 'Budget', 'Runtime']

specific_data = l.get_imdb(target_movies, target_fields)

print("Custom Extraction Results:")
print(f"{'ID':<5} | {'Director':<20} | {'Budget':<12} | {'Runtime'}")
print("-" * 60)

for row in specific_data:
    # row is [movieId, Field1, Field2, Field3]
    mid, director, budget, runtime = row
    print(f"{mid:<5} | {director:<20} | ${budget:<11} | {runtime}")

Custom Extraction Results:
ID    | Director             | Budget       | Runtime
------------------------------------------------------------
2     | Joe Johnston         | $65000000    | 104 min
1     | John Lasseter        | $30000000    | 81 min


## The Director's Cut
### Question: Who are the most prolific directors? (Scraped live from IMDb!)

In [4]:
%timeit
# Note: We limit this to top 3 to avoid hitting IMDb rate limits during the presentation
directors = l.top_directors(3)
print("Top Directors in this sample:", directors)

Top Directors in this sample: {'John Lasseter': 1, 'Joe Johnston': 1, 'Howard Deutch': 1}


## The Big Spenders
### Question: Which movies had the biggest production budgets?
We scraped IMDb to find the most expensive movies in our sample.

In [5]:
%timeit
# Top 5 most expensive movies
expensive = l.most_expensive(5)
print("Most Expensive Movies:")
for title, budget in expensive.items():
    print(f"{title}: ${budget:,}")

Most Expensive Movies:
Jumanji (1995): $65,000,000
Heat (1995): $60,000,000
GoldenEye (1995): $60,000,000
Sabrina (1995): $58,000,000
Sudden Death (1995): $35,000,000


## Return on Investment
### Question: Which movies were the most profitable (Gross - Budget)?

In [6]:
%timeit
# Top 5 profitable
profitable = l.most_profitable(5)
print("Most Profitable Movies:")
for title, profit in profitable.items():
    print(f"{title}: ${profit:,}")

Most Profitable Movies:
Toy Story (1995): $371,157,969
GoldenEye (1995): $292,194,034
Jumanji (1995): $197,821,940
Heat (1995): $127,436,818
Waiting to Exhale (1995): $65,452,156


## Marathon Movies
### Question: Which movies require the most popcorn (longest runtime)?

In [7]:
%timeit
# Top 5 longest
longest = l.longest(5)
print("Longest Movies:")
for title, runtime in longest.items():
    print(f"{title}: {runtime}")

Longest Movies:
Heat (1995): 170 min
GoldenEye (1995): 130 min
Sabrina (1995): 127 min
Waiting to Exhale (1995): 124 min
Sudden Death (1995): 111 min


## Cost per Minute
### Question: Which movies cost the most to film per minute of screen time?
This metric tells us about the density of production value.

In [8]:
%timeit
# Top 5 cost per minute
cpm = l.top_cost_per_minute(5)
print("Highest Cost Per Minute:")
for title, cost in cpm.items():
    print(f"{title}: ${cost:,.2f} per minute")

Highest Cost Per Minute:
Jumanji (1995): $625,000.00 per minute
GoldenEye (1995): $461,538.46 per minute
Sabrina (1995): $456,692.91 per minute
Toy Story (1995): $370,370.37 per minute
Heat (1995): $352,941.18 per minute


## Box Office Hits (Bonus)
### Question: Which movies made the most money worldwide?

In [9]:
%timeit
# Top 5 grossing movies (Bonus Method)
grossing = l.top_grossing(5)
print("Top Grossing Movies:")
for title, rev in grossing.items():
    print(f"{title}: ${rev:,}")

Top Grossing Movies:
Toy Story (1995): $401,157,969
GoldenEye (1995): $352,194,034
Jumanji (1995): $262,821,940
Heat (1995): $187,436,818
Waiting to Exhale (1995): $81,452,156


In [10]:
%timeit
# Bonus method usage
star_movies = m.search_by_title("Star")
print(f"Found {len(star_movies)} movies matching 'Star':")
for mid, title in star_movies.items():
    print(f"ID {mid}: {title}")

Found 6 movies matching 'Star':
ID 260: Star Wars: Episode IV - A New Hope (1977)
ID 316: Stargate (1994)
ID 329: Star Trek: Generations (1994)
ID 800: Lone Star (1996)
ID 1196: Star Wars: Episode V - The Empire Strikes Back (1980)
ID 1210: Star Wars: Episode VI - Return of the Jedi (1983)


# Movies

## 📅 Temporal Analysis: The Golden Age of Cinema
### Question: How has movie production volume changed over time?
We analyze the `dist_by_release` method to identify which years in our dataset had the highest number of movie releases. Is the 90s really the peak of this sample?

In [11]:
%timeit
# Analyze release years
release_data = m.dist_by_release()
top_year = list(release_data.keys())[0]
top_count = list(release_data.values())[0]

print(f"🏆 The peak year is {top_year} with {top_count} movies.")
print("\nTop 5 Years:")
for year, count in list(release_data.items())[:5]:
    print(f"{year}: {count} releases")

🏆 The peak year is 1995 with 224 movies.

Top 5 Years:
1995: 224 releases
1994: 184 releases
1996: 181 releases
1993: 101 releases
1992: 23 releases


## 🎭 Genre Dominance: What defines the industry?
### Question: Which genres are the most common?
By parsing the pipe-separated genre strings (e.g., `Adventure|Children|Fantasy`), we calculate the frequency of every genre in the dataset using `dist_by_genres`.

In [12]:
%timeit
# Analyze genres
genre_data = m.dist_by_genres()

print("Top 5 Most Common Genres:")
print("-" * 30)
for genre, count in list(genre_data.items())[:5]:
    print(f"{genre:<15} | {count}")

Top 5 Most Common Genres:
------------------------------
Drama           | 507
Comedy          | 365
Romance         | 208
Thriller        | 179
Action          | 158


## 🧬 Cinematic Complexity: The "Jack of All Trades"
### Question: Which movies fit into the most categories at once?
Some movies are simple "Comedies". Others are "Action|Adventure|Sci-Fi|Thriller". We use `most_genres` to find the movies with the widest range of classifications.

In [13]:
%timeit
# Find movies with the most genre tags
complex_movies = m.most_genres(5)

print("Movies with the highest genre count:")
for title, count in complex_movies.items():
    print(f"{title}: {count} genres")

Movies with the highest genre count:
Strange Days (1995): 6 genres
Lion King, The (1994): 6 genres
Getaway, The (1994): 6 genres
Super Mario Bros. (1993): 6 genres
Beauty and the Beast (1991): 6 genres


## 🔍 Interactive Tool: Movie Search (Bonus)
### Question: Can we find specific movies in the dataset?
Using our custom `search_by_title` method, we can filter the dataset by keyword. Let's look for any movie containing the word **"Love"**.

In [14]:
%timeit
# Search for movies containing "Love"
keyword = "Love"
search_results = m.search_by_title(keyword)

print(f"Found {len(search_results)} movies matching '{keyword}':")
print("-" * 40)
# Showing first 10 results to keep the report clean
for mid, title in list(search_results.items())[:10]:
    print(f"[{mid}] {title}")

Found 21 movies matching 'Love':
----------------------------------------
[40] Cry, the Beloved Country (1995)
[106] Nobody Loves Me (Keiner liebt mich) (1994)
[178] Love & Human Remains (1993)
[179] Mad Love (1995)
[249] Immortal Beloved (1994)
[270] Love Affair (1994)
[287] Nina Takes a Lover (1994)
[295] Pyromaniac's Love Story, A (1995)
[360] I Love Trouble (1994)
[381] When a Man Loves a Woman (1994)


# Ratings

## 📉 Rating History: When were users most active?
### Question: How has the volume of ratings changed over the years?
We look at `dist_by_year` to see the timeline of user engagement.

In [15]:
%timeit
# Distribution of ratings by year
history = r.movies.dist_by_year()

print("Rating Activity by Year:")
print("-" * 30)
for year, count in history.items():
    print(f"{year}: {count} ratings")

Rating Activity by Year:
------------------------------
1996: 358 ratings
1999: 82 ratings
2000: 296 ratings
2001: 70 ratings
2005: 121 ratings
2006: 4 ratings
2007: 1 ratings
2011: 39 ratings
2015: 29 ratings


## ⭐ The Star System: How do people rate?
### Question: Do users prefer giving 5 stars or are they critical?
Using `dist_by_rating`, we analyze the frequency of each rating value (0.5 to 5.0).

In [16]:
%timeit
# Distribution of specific rating values
stars_dist = r.movies.dist_by_rating()

print("Star Distribution:")
for stars, count in stars_dist.items():
    # Create a simple visual bar using asterisks
    bar = "*" * (count // 10) 
    print(f"{stars:>3} stars: {count:<5} | {bar}")

Star Distribution:
0.5 stars: 24    | **
1.0 stars: 39    | ***
1.5 stars: 11    | *
2.0 stars: 57    | *****
2.5 stars: 7     | 
3.0 stars: 253   | *************************
3.5 stars: 17    | *
4.0 stars: 292   | *****************************
4.5 stars: 33    | ***
5.0 stars: 267   | **************************


## 🗳️ The Popular Vote
### Question: Which movies received the most attention (most ratings)?
Using `top_by_num_of_ratings`, we find the movies that were watched and rated by the most users.

In [17]:
%timeit
# Top 5 movies by volume of ratings
most_rated = r.movies.top_by_num_of_ratings(5)

print("Most Rated Movies:")
for title, count in most_rated.items():
    print(f"{title}: {count} ratings")

Most Rated Movies:
Usual Suspects, The (1995): 4 ratings
Pulp Fiction (1994): 4 ratings
Fugitive, The (1993): 4 ratings
Schindler's List (1993): 4 ratings
Batman (1989): 4 ratings


## 🏆 Critical Acclaim: The Best Movies
### Question: Which movies have the highest average score?
Using `top_by_ratings`, we find the audience favorites.

In [18]:
%timeit
# Top 5 movies by average rating
best_rated = r.movies.top_by_ratings(5, metric='average')

print("Highest Rated Movies (Average):")
for title, score in best_rated.items():
    print(f"{title}: {score} stars")

Highest Rated Movies (Average):
Bottle Rocket (1996): 5.0 stars
Canadian Bacon (1995): 5.0 stars
Star Wars: Episode IV - A New Hope (1977): 5.0 stars
James and the Giant Peach (1996): 5.0 stars
Wizard of Oz, The (1939): 5.0 stars


## 💔 Love it or Hate it: The Controversial Movies
### Question: Which movies divide the audience?
High variance means opinions are split (some give 1 star, others give 5). We use `top_controversial` to find them.

In [19]:
%timeit
# Top 5 movies by variance
divisive = r.movies.top_controversial(5)

print("Most Controversial Movies (High Variance):")
for title, var in divisive.items():
    print(f"{title}: {var} variance")

Most Controversial Movies (High Variance):
2018: 10.12 variance
2090: 10.12 variance
My Fair Lady (1964): 10.12 variance
2571: 8.0 variance
1704: 6.12 variance


## 👤 User Behavior: The Power Users
### Question: Who contributes the most data?
We analyze `dist_by_ratings_number` to find the most active users in the dataset.

In [20]:
%timeit
# Top 5 active users
active_users = r.users.dist_by_ratings_number()

print("Top 5 Most Active Users:")
# Slice the first 5 items from the dict
for user_id, count in list(active_users.items())[:5]:
    print(f"User ID {user_id}: {count} ratings")

Top 5 Most Active Users:
User ID 6: 314 ratings
User ID 1: 232 ratings
User ID 4: 216 ratings
User ID 7: 126 ratings
User ID 5: 44 ratings


## 🎁 The Critics vs. The Fans
### Question: Who are the kindest users? And who are the harshest critics?
We look at `dist_by_ratings_average` to see the average score given by each user.

In [21]:
%timeit
# Top 5 "Kindest" users (High average rating)
generous_users = r.users.dist_by_ratings_average()

print("Users with highest average ratings:")
for user_id, avg in list(generous_users.items())[:5]:
    print(f"User ID {user_id}: avg rating {avg:.2f}")

Users with highest average ratings:
User ID 1: avg rating 4.37
User ID 2: avg rating 3.95
User ID 5: avg rating 3.64
User ID 4: avg rating 3.56
User ID 6: avg rating 3.49


## 🎢 Emotional Range: User Volatility
### Question: Which users have the wildest swings in their ratings?
Using `dist_by_ratings_variance`, we find users who use the full range of the star scale versus those who rate everything similarly.

In [22]:
%timeit
# Top 5 users with highest variance
volatile_users = r.users.dist_by_ratings_variance()

print("Users with most diverse ratings (High Variance):")
for user_id, var in list(volatile_users.items())[:5]:
    print(f"User ID {user_id}: variance {var:.2f}")

Users with most diverse ratings (High Variance):
User ID 3: variance 4.37
User ID 4: variance 1.73
User ID 7: variance 1.67
User ID 5: variance 0.98
User ID 6: variance 0.72


# Tags

## 🗣️ The Word on the Street: Popular Tags
### Question: What are the most common labels users apply to movies?
We use `most_popular` to find the most frequent one-word descriptions in the dataset.

In [23]:
%timeit
# Top 5 most used tags
popular = t.most_popular(5)

print("Most Used Tags:")
print("-" * 20)
for tag, count in popular.items():
    print(f"{tag:<20} | {count} times")

Most Used Tags:
--------------------
funny                | 15 times
sci-fi               | 14 times
twist ending         | 12 times
dark comedy          | 12 times
atmospheric          | 10 times


## 📏 Short Story vs. Novel
### Question: What are the longest tags by character count?
Some users leave simple tags like "funny". Others write whole sentences. We use `longest` to find the latter.

In [24]:
%timeit
# Top 5 longest tags by characters
long_tags = t.longest(5)

print("Longest Tags (Characters):")
for tag in long_tags:
    print(f"[{len(tag)} chars] {tag}")

Longest Tags (Characters):
[85 chars] Something for everyone in this one... saw it without and plan on seeing it with kids!
[63 chars] the catholic church is the most corrupt organization in history
[36 chars] audience intelligence underestimated
[35 chars] Oscar (Best Music - Original Score)
[28 chars] assassin-in-training (scene)


## 📝 Verbosity: Tags with the most words
### Question: Which tags contain the highest number of individual words?
Using `most_words`, we look for tags that are phrases or sentences rather than single keywords.

In [25]:
%timeit
# Top 5 tags with most words
verbose_tags = t.most_words(5)

print("Tags with most words:")
for tag, count in verbose_tags.items():
    print(f"({count} words) {tag}")

Tags with most words:
(16 words) Something for everyone in this one... saw it without and plan on seeing it with kids!
(10 words) the catholic church is the most corrupt organization in history
(6 words) Oscar (Best Music - Original Score)
(5 words) Everything you want is here
(5 words) based on a true story


## 🔗 The Intersection
### Question: Which tags are BOTH long (chars) AND verbose (words)?
We use `most_words_and_longest` to find tags that appear in the top lists of both previous categories.

In [26]:
%timeit
# Intersection of top 50 longest and top 50 verbose
# We use a larger number (50) to ensure we find an overlap in this small dataset
intersection = t.most_words_and_longest(50)

print(f"Found {len(intersection)} tags in the intersection:")
for tag in intersection:
    print(f"- {tag}")

Found 26 tags in the intersection:
- Guardians of the Galaxy
- Oscar (Best Actress)
- the catholic church is the most corrupt organization in history
- heroine in tight suit
- video game adaptation
- helena bonham carter
- Oscar (Best Cinematography)
- freedom of expression
- Something for everyone in this one... saw it without and plan on seeing it with kids!
- mindless one liners
- Everything you want is here
- jay and silent bob
- Oscar (Best Music - Original Score)
- audience intelligence underestimated
- Nudity (Full Frontal)
- seen more than once
- Disney animated feature
- multiple short stories
- based on a true story
- indie record label
- political right versus left
- Harley Quinn's ass
- ROBOTS AND ANDROIDS
- sexy female scientist
- representation of children
- Francis Ford Coppola


## 🔍 Tag Search
### Question: Can we find all variants of a specific tag topic?
Using `tags_with`, we search for every tag that contains the word **"funny"**.

In [27]:
%timeit
# Search for tags containing 'funny'
funny_tags = t.tags_with('funny')

print("Tags containing 'funny':")
print(funny_tags)

Tags containing 'funny':
['funny', 'stupid but funny', 'very funny']


## 🧠 Tag Complexity (Bonus)
### Question: What is the average length of a word used in tags?
Are users using simple words or complex vocabulary? We calculate the average word length across all tags.

In [28]:
%timeit
# Calculate average word length
avg_len = t.avg_word_length()
print(f"The average word length in tags is: {avg_len} characters")

The average word length in tags is: 6.37 characters
